<a href="https://colab.research.google.com/github/duncansnh/Bare-peat/blob/master/Indices_creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Script to create indices from ARD of 10 bands from Sentinel 2. Creates 23 band raster (float 32). (note - If the number of indices is altered, count= will need updating. ) Forms basis of supervised classification for bare peat mapping. December 2019.
     

In [0]:
#This is only required if running in colab notebook to install the libraries
#If running Python code elsewhere need to make sure below libraries are installed
! pip install geopandas
! pip install descartes
! pip install rasterio
! pip install rasterstats

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import json
from sklearn.model_selection import train_test_split
import geopandas as gpd
import descartes
import rasterio
from rasterio.mask import mask
from rasterio.features import geometry_mask
from shapely.geometry import mapping
from rasterstats import zonal_stats
import datetime

In [0]:
#Only if running in Google Colab, in which case input image, training polygons and output results need to be in Google Drive.
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Open/ create log file, set working drive.

In [0]:

wd = '/content/drive/My Drive'
iteration = 'ML14'
logname = iteration + '.txt'
filename= os.path.join(wd,'Log_files', logname)
file_object = open(filename,"x")

###Set image directory

In [0]:
image_dir = os.path.join(wd, 'Imagery')



###Start timer log.

In [0]:
starttime1 = datetime.datetime.now()

###'open' input 10 band raster, update profile for output

In [0]:
#Read image
s2 = rasterio.open(os.path.join(image_dir,'S2_MLarea14.tif'))
#Print number of bands
B = s2.count
print(B)
print(s2.shape)
#Copy raster profile to for later output
s2prof = s2.profile.copy()


#calculate 23 indices for whole image - windowed read/ write

In [0]:

#output greater than 4GB therefore needs to be BigTIFF
s2prof.update(count=23, nodata=None, dtype=np.float32, BIGTIFF="IF_SAFER")
print((s2prof))

with rasterio.open(os.path.join(image_dir,'{}_23_indices.tif'.format(iteration)), 'w', **s2prof) as dst:
  for block_index, window in dst.block_windows(1):
    #read in each band for current window
    s2_block_blue = s2.read(1, window=window, masked=True).astype(float)
    s2_block_green = s2.read(2, window=window, masked=True).astype(float)
    s2_block_red = s2.read(3, window=window, masked=True).astype(float)
    s2_block_re5 = s2.read(4, window=window, masked=True).astype(float)
    s2_block_re6 = s2.read(5, window=window, masked=True).astype(float)
    s2_block_re7 = s2.read(6, window=window, masked=True).astype(float)
    s2_block_NIR = s2.read(7, window=window, masked=True).astype(float)
    s2_block_NIR8A = s2.read(8, window=window, masked=True).astype(float)
    s2_block_SWIR1 = s2.read(9, window=window, masked=True).astype(float)
    s2_block_SWIR2 = s2.read(10, window=window, masked=True).astype(float)
    
    v = s2_block_red.shape
    print((v))
    print((s2_block_red[0,100]))
    np.seterr(divide='ignore', invalid = 'ignore' ) #ignore errors from calculating indices/ ratios#over= 'ignore', under = 'ignore' 

    #calculate indices and convert to float
    ndvi = ((s2_block_NIR - s2_block_red)/(s2_block_NIR + s2_block_red))
    ndvi = ndvi.astype(rasterio.float32)
    Gndvi=((s2_block_NIR - s2_block_green)/(s2_block_NIR + s2_block_green))
    Gndvi= Gndvi.astype(rasterio.float32)
    Bndvi=((s2_block_NIR - s2_block_blue)/(s2_block_NIR + s2_block_blue)).astype(rasterio.float32)
    Re5ndvi=((s2_block_re5 - s2_block_red)/(s2_block_re5 + s2_block_red)).astype(rasterio.float32)
    Re6ndvi=((s2_block_re6 - s2_block_red)/(s2_block_re6 + s2_block_red)).astype(rasterio.float32)
    Re7ndvi=((s2_block_re7 - s2_block_red)/(s2_block_re7 + s2_block_red)).astype(rasterio.float32)
    Re8Andvi=((s2_block_NIR8A - s2_block_red)/(s2_block_NIR8A + s2_block_red)).astype(rasterio.float32)
    Re6Gndvi=((s2_block_re6 - s2_block_green)/(s2_block_re6 + s2_block_green)).astype(rasterio.float32)
    Re7Gndvi=((s2_block_re7 - s2_block_green)/(s2_block_re7 + s2_block_green)).astype(rasterio.float32)
    Re8AGndvi=((s2_block_NIR8A - s2_block_green)/(s2_block_NIR8A + s2_block_green)).astype(rasterio.float32)
    NDWI=((s2_block_green - s2_block_NIR)/(s2_block_green + s2_block_NIR)).astype(rasterio.float32)
    mDNWI=((s2_block_green - s2_block_SWIR1)/(s2_block_green + s2_block_SWIR1)).astype(rasterio.float32)
    mNDVI=((s2_block_re7 - s2_block_re5)/(s2_block_re7 + s2_block_re5)).astype(rasterio.float32)
    darkness=((s2_block_blue+s2_block_green+s2_block_red)/3).astype(rasterio.float32)
    ratioblueSWIR1= (s2_block_blue/s2_block_SWIR1).astype(rasterio.float32)
    ratioNIRgreen=(s2_block_NIR/s2_block_green).astype(rasterio.float32)
    ratioredSWIR1=(s2_block_red/s2_block_SWIR1).astype(rasterio.float32)
    ratioRe5SWIR2=(s2_block_re5/s2_block_SWIR2).astype(rasterio.float32)
    ratioRe5SWIR1=(s2_block_re5/s2_block_SWIR1).astype(rasterio.float32)
    ratiogreenSWIR1=(s2_block_green/s2_block_SWIR1).astype(rasterio.float32)
    ratioRe7SWIR1=(s2_block_re7/s2_block_SWIR1).astype(rasterio.float32)
    ratioRe8ASWIR1=(s2_block_NIR8A/s2_block_SWIR1).astype(rasterio.float32)
    ratioRe6SWIR1=(s2_block_re6/s2_block_SWIR1).astype(rasterio.float32)

    print((ratioRe8ASWIR1[0,100]))
    print((window))
    #write results for current window
    dst.write_band(1, ndvi, window = window)
    dst.write_band(2, Gndvi, window = window)
    dst.write_band(3, Bndvi, window = window)
    dst.write_band(4, Re5ndvi, window = window)
    dst.write_band(5, Re6ndvi, window = window)
    dst.write_band(6, Re7ndvi, window = window)
    dst.write_band(7, Re8Andvi, window = window)
    dst.write_band(8, Re6Gndvi, window = window)
    dst.write_band(9, Re7Gndvi, window = window)
    dst.write_band(10, Re8AGndvi, window = window)
    dst.write_band(11, NDWI, window = window)
    dst.write_band(12, mDNWI, window = window)
    dst.write_band(13, mNDVI, window = window)
    dst.write_band(14, darkness, window = window)
    dst.write_band(15, ratioblueSWIR1, window = window)
    dst.write_band(16, ratioNIRgreen, window = window)
    dst.write_band(17, ratioredSWIR1, window = window)
    dst.write_band(18, ratioRe5SWIR2, window = window)
    dst.write_band(19, ratioRe5SWIR1, window = window)
    dst.write_band(20, ratiogreenSWIR1, window = window)
    dst.write_band(21, ratioRe7SWIR1, window = window)
    dst.write_band(22, ratioRe8ASWIR1, window = window)
    dst.write_band(23, ratioRe6SWIR1, window = window)


record time taken 

In [0]:
endtime1=datetime.datetime.now()
deltatime1=endtime1-starttime1
print ((deltatime1))
print(("Time to read in S2 image:  {0}  hr:min:sec".format(deltatime1))) # jupyter notebook records time taken
file_object.write("Time to read in S2 image:  {0}  hr:min:sec".format(deltatime1))                        


Visualise image, check output


In [0]:
from matplotlib import pyplot
plotraster = rasterio.open(os.path.join(image_dir,'{}_23_indices.tif'.format(iteration)))

pyplot.imshow(plotraster.read(14),cmap='pink')
pyplot.show()

#img = plotraster.read(10)
#print((img.shape))


from rasterio.plot import show

show(plotraster.read(10))

plotraster.close()
